# Final Report

In [ ]:
import json
import pickle
import numpy as np
import pandas as pd
import datetime as dt
from scipy.stats import t
from scipy import interp
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier

plt.style.use('ggplot')

In [ ]:
notebook_config = json.load(open('../config/notebook.json', 'r'))
if notebook_config['testing']:
    data_dir = '../test/'
else:
    data_dir = '../data/'

# 1. Result Analysis

In [ ]:
test_data = pd.read_pickle(data_dir + 'tmp/model_results.pkl')

## 1.1 Accuracy

In [ ]:
print('baseline accuracy: ' + str(sum(test_data['target'] == test_data['base_pred']) / len(test_data['target'])))
print('unigram accuracy: ' + str(sum(test_data['target'] == test_data['unigram_pred']) / len(test_data['target'])))
print('phrase accuracy: ' + str(sum(test_data['target'] == test_data['phrase_pred']) / len(test_data['target'])))

- **Note**:
    - Adding text info did improve the performance of the models
    - Phrase models and unigram models are not as different as we expected, maybe because there are many high-quality unigrams as well.

In [ ]:
up_data = test_data.loc[test_data['phrase_pred'] == 'UP'].copy()
stay_data = test_data.loc[test_data['phrase_pred'] == 'STAY'].copy()
down_data = test_data.loc[test_data['phrase_pred'] == 'DOWN'].copy()

In [ ]:
labels = [up_data, stay_data, down_data]
models = ['base_pred', 'unigram_pred', 'phrase_pred']
accuracy = []
for model in models:
    temp = []
    for label in labels:
        if len(label) == 0:
            temp.append(0)
        else:
            temp.append(len(label[label['target'] == label[model]]) / len(label))
    accuracy.append(temp)

In [ ]:
pd.DataFrame({'baseline': accuracy[0],
             'unigram': accuracy[1],
             'phrase': accuracy[2]},
            index = ['UP', 'STAY', 'DOWN'] )

- **Note**: Our phrase model performs much better for the `STAY` class compared to other baselines!

# 2. Feature Importance

In [ ]:
data = pd.read_pickle(data_dir + 'processed/feature_encoded_merged_data.pkl')

train = data.loc[data['dataset'] == 'train'].copy()
val = data.loc[data['dataset'] == 'val'].copy()
test = data.loc[data['dataset'] == 'test'].copy()

unigrams = pd.read_csv(data_dir + 'processed/model_unigrams.csv')
phrases = pd.read_csv(data_dir + 'financial_phrases_sample.txt', sep = '\t', header = None)

In [ ]:
uni_model = pickle.load(open(data_dir + 'models/uni_model', 'rb'))
phrase_model = pickle.load(open(data_dir + 'models/phrase_model', 'rb'))

In [ ]:
mlb = MultiLabelBinarizer()

all_events = pd.DataFrame(mlb.fit_transform(data['cleaned_event']),
                   columns = mlb.classes_,
                   index = data['cleaned_event'].index)

In [ ]:
events = np.array(all_events.columns)
numerical = np.array(['Surprise(%)', 'price_change_7', 
              'price_change_30', 'price_change_90', 'price_change_365',
              'prev_vix_values'])

In [ ]:
uni_importance = uni_model.feature_importances_
phrase_importance = phrase_model.feature_importances_

## 2.1 Unigram Model Feature Importance

In [ ]:
uni_feature_importance = pd.DataFrame({'feature': np.concatenate((events, numerical, unigrams.values.ravel())),
                                       'importance': uni_importance})
uni_feature_importance.sort_values(by = 'importance', ascending = False).head(20).reset_index(drop = True)

## 2.2 Phrase Model Feature Importance

In [ ]:
if len(phrases) > len(phrase_importance):
    phrase_features = np.concatenate((events, numerical, phrases[1].values.ravel()))[:len(phrase_importance)]
else:
    phrase_importance = phrase_importance[:len(phrases)]

phrase_feature_importance = pd.DataFrame({'feature': phrase_features,
                                       'importance': phrase_importance})
phrase_feature_importance.sort_values(by = 'importance', ascending = False).head(20).reset_index(drop = True)

- **Summary**: As we can see from the two tables above, the important features in our model, compared to those in the unigram model, are relatively similar. In fact, all of the text features have not make it into top 5 :/